# **Project Requirements**

## **반드시 구현되어야 하는 기능**

### **1. 데이터 가져오기(Pull)**

여러분이 수집하는 데이터를 정하는 것부터 시작합니다. 데이터는 다양한 방법으로 존재할 수 있습니다. 아래는 작업의 예시입니다. 데이터를 수집하는 작업을 진행해야합니다.

- 파일 다운로드 및 파일 읽기
- 데이터 스크레이핑 & 크롤링
- API 를 이용한 데이터 수집

Advanced

- 일정 시간 간격으로 데이터 수집(cron, 윈도우 스케줄링, APScheduler …)
- 동적 스크레이핑 (셀레니움 … )

---

### **2. 데이터 저장(Store)**

1.에서 가져온 데이터를 영원히 여러분의 메모리에 남길 수 없습니다. 가져온 데이터를 데이터베이스에 저장해야합니다.

- 관계형 데이터베이스 (sqlite, postgreSQL …)
- NoSQL 데이터베이스 (MongoDB … )

Advanced

- 로컬 데이터베이스를 배포해서 사용하는 방법

---

### **3. API 서비스 개발 (Service)**

수집된 데이터베이스의 데이터를 기반으로 Section 2 에서 배운 모델을 구성합니다. 먼저, 모델의 사용은 localhost API 로 개발해서 작업증명(Proof-of-Concept) 를 진행합니다.

- 개발한 모델이 API 로 사용가능해야 합니다.

Advanced

- API 를 다른 개발자들이 사용할 수 있도록 배포
- 서비스 사용자의 API 사용을 도와주기 위해, GUI 를 구성합니다. (부트스트랩 …)
- API 사용 결과가 데이터베이스에 저장될 수 있도록 변경

---

### **4. 데이터 분석용 대시보드 개발**

데이터베이스의 데이터를 기반으로 대시보드에 자신의 의견을 피력하기 위한 그래프를 구성합니다. 데이터 분석은 대시보드를 통해 진행되어야 하며, 만들어야할 그래프의 최소 개수는 3개 입니다.

- 데이터베이스의 데이터를 기반으로 EDA 를 진행합니다.
- 구글 데이터스튜디오와 metabase 는 기본 점수입니다.

Advanced

- `3.API 서비스 개발 (Service)` 의 모델 결과가 EDA 에 포함되도록 그래프를 구성합니다.
- Redash 를 이용한 대시보드 개발
- 대시보드 배포

# **서울대공원 동/식물원 입장객 수 예측 API 서비스**





## **EDA**

In [1]:
import pandas as pd
import datetime

In [2]:
### 2004년 
def make_df2(year, month): 
    excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
    month = f'{month}월' 
    df = pd.read_excel(excel, sheet_name = month)
    df = pd.concat([df.iloc[5:-1, :2], df.iloc[5:-1, 14]], axis=1)
    df.columns = ['날짜','요일', '총 입장객 수']
    df.reset_index(inplace=True, drop=True)
    globals()['df_{}_{}'.format(year, month)] = df
    return globals()['df_{}_{}'.format(year, month)]

# 해당 년도 데이터 합치기
df_2004 = pd.concat([make_df2(2004, 7), make_df2(2004, 8), make_df2(2004, 9), make_df2(2004, 10), make_df2(2004, 11), make_df2(2004, 12)])
df_2004['날짜'] = '2004년' + df_2004['날짜']
# print(len(df_2004))
# df_2004



### 2005년 ~ 2013년
for i in range(2005, 2014):
    globals()['df_{}'.format(i)] = pd.concat([make_df2(i, 1), make_df2(i, 2), make_df2(i, 3), make_df2(i, 4), make_df2(i, 5), make_df2(i, 6), make_df2(i, 7), make_df2(i, 8), make_df2(i, 9), make_df2(i, 10), make_df2(i, 11), make_df2(i, 12)])
    globals()['df_{}'.format(i)]['날짜'] = f'{i}년' + globals()['df_{}'.format(i)]['날짜']
    # print(f'{i}년 : ', len(globals()['df_{}'.format(i)]))
    df_2004 = pd.concat([df_2004, globals()['df_{}'.format(i)]])

# print(df_2004)

### 2014년
df_2014 = pd.concat([make_df2(2014, 1), make_df2(2014, 3), make_df2(2014, 4), make_df2(2014, 5), make_df2(2014, 6), make_df2(2014, 7), make_df2(2014, 8), make_df2(2014, 9), make_df2(2014, 10), make_df2(2014, 11), make_df2(2014, 12)])
df_2014['날짜'] = '2014년' + df_2014['날짜']

df = pd.concat([df_2004, df_2014])

# print(df)

### date
def datechange(date):
    x = datetime.datetime.strptime(date, "%Y년%m월%d일").date()
    return x

df['날짜'] = df['날짜'].apply(datechange)
df.reset_index(inplace=True, drop=True)
df

,날짜,요일,총 입장객 수
0,2004-07-01,목,1695
1,2004-07-02,금,1240
2,2004-07-03,토,3852
3,2004-07-04,일,694
4,2004-07-05,월,1545
...,...,...,...
3774,2014-12-27,토,3572
3775,2014-12-28,일,3506
3776,2014-12-29,월,1705
3777,2014-12-30,화,1663


In [3]:
#### 2015년 ~ 2020년 : 데이터 입력 형식 다름
def make_df2(year, month): 
    excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
    month = f'{month}월' 
    df = pd.read_excel(excel, sheet_name = month)
    df = pd.concat([df.iloc[5:-1, :2], df.iloc[5:-1, 14]], axis=1)
    df.columns = ['날짜', '요일', '총 입장객 수']
    df.reset_index(inplace=True, drop=True)
    globals()['df_{}_{}'.format(year, month)] = df
    return globals()['df_{}_{}'.format(year, month)]



df_2015 = pd.concat([make_df2(2015, 1), make_df2(2015, 2), make_df2(2015, 3), make_df2(2015, 4), make_df2(2015, 5), make_df2(2015, 6), make_df2(2015, 7), make_df2(2015, 8), make_df2(2015, 9), make_df2(2015, 10), make_df2(2015, 11), make_df2(2015, 12)])
df_2016 = pd.concat([make_df2(2016, 1), make_df2(2016, 2), make_df2(2016, 3), make_df2(2016, 4), make_df2(2016, 5), make_df2(2016, 6), make_df2(2016, 7), make_df2(2016, 8), make_df2(2016, 9), make_df2(2016, 10), make_df2(2016, 11), make_df2(2016, 12)])
df_2017 = pd.concat([make_df2(2017, 3), make_df2(2017, 4), make_df2(2017, 5), make_df2(2017, 6), make_df2(2017, 7), make_df2(2017, 8), make_df2(2017, 9), make_df2(2017, 10), make_df2(2017, 11), make_df2(2017, 12)])
df_2018 = pd.concat([make_df2(2018, 1), make_df2(2018, 2), make_df2(2018, 3), make_df2(2018, 4), make_df2(2018, 5), make_df2(2018, 6), make_df2(2018, 7), make_df2(2018, 8), make_df2(2018, 9), make_df2(2018, 10), make_df2(2018, 11), make_df2(2018, 12)])
df_2019 = pd.concat([make_df2(2019, 1), make_df2(2019, 2), make_df2(2019, 3), make_df2(2019, 4), make_df2(2019, 5), make_df2(2019, 6), make_df2(2019, 7), make_df2(2019, 8), make_df2(2019, 9), make_df2(2019, 10), make_df2(2019, 11), make_df2(2019, 12)])
df_2020 = pd.concat([make_df2(2020, 1), make_df2(2020, 2), make_df2(2020, 3), make_df2(2020, 4)])

### date
df2 = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019, df_2020])
df2['날짜'] = pd.to_datetime(df2['날짜'])

df2

,날짜,요일,총 입장객 수
0,2015-01-03,토,3314
1,2015-01-04,일,5201
2,2015-01-05,월,1578
3,2015-01-06,화,1255
4,2015-01-07,수,1006
...,...,...,...
23,2020-04-25,토,7299
24,2020-04-26,일,11180
25,2020-04-27,월,2104
26,2020-04-28,화,2709


In [4]:
#### final_df
final_df = pd.concat([df, df2])
final_df.reset_index(inplace=True, drop=True)


### 요일 > 주말 여부 (yes:1, no:0)
for i in range(len(final_df)):
    if final_df['요일'][i] == '토':
        final_df['요일'][i] = 1
    elif final_df['요일'][i] == '일':
        final_df['요일'][i] = 1
    else:
        final_df['요일'][i] = 0
final_df.rename(columns = {'요일':'주말 여부'}, inplace=True)


### 공휴일 여부 (yes:1, no:0)
holiday = []



final_df['날짜'] = pd.to_datetime(final_df['날짜'])
final_df.set_index('날짜', inplace=True)
final_df = final_df.astype(int)
final_df.reset_index(inplace=True)
final_df

,날짜,주말 여부,총 입장객 수
0,2004-07-01,0,1695
1,2004-07-02,0,1240
2,2004-07-03,1,3852
3,2004-07-04,1,694
4,2004-07-05,0,1545
...,...,...,...
5491,2020-04-25,1,7299
5492,2020-04-26,1,11180
5493,2020-04-27,0,2104
5494,2020-04-28,0,2709


In [5]:
### merge 기온 to final_df
temperature = pd.read_excel('/content/temperature.xlsx', engine='openpyxl')
temperature.fillna(value=0, inplace=True)
final_df = pd.merge(final_df, temperature, left_on='날짜', right_on='날짜', how='left')
final_df

,날짜,주말 여부,총 입장객 수,평균기온(℃),최저기온(℃),최고기온(℃)
0,2004-07-01,0,1695,24.0,22.0,26.7
1,2004-07-02,0,1240,23.1,21.3,24.2
2,2004-07-03,1,3852,24.4,22.0,26.8
3,2004-07-04,1,694,21.1,19.7,22.7
4,2004-07-05,0,1545,22.8,19.5,26.2
...,...,...,...,...,...,...
5491,2020-04-25,1,7299,13.1,9.9,17.1
5492,2020-04-26,1,11180,12.3,6.8,19.0
5493,2020-04-27,0,2104,12.9,8.7,18.3
5494,2020-04-28,0,2709,13.9,7.8,19.9


In [6]:
### 강수량 : null값 채우기
rainfall = pd.read_excel('/content/rainfall.xlsx', engine='openpyxl')
rainfall.fillna(value=0, inplace=True)
# rainfall.isnull().sum()

### merge 강수량 to final_df
final_df = pd.merge(final_df, rainfall, left_on='날짜', right_on='날짜', how='left')
final_df

,날짜,주말 여부,총 입장객 수,평균기온(℃),최저기온(℃),최고기온(℃),강수량(mm)
0,2004-07-01,0,1695,24.0,22.0,26.7,4.5
1,2004-07-02,0,1240,23.1,21.3,24.2,10.5
2,2004-07-03,1,3852,24.4,22.0,26.8,10.5
3,2004-07-04,1,694,21.1,19.7,22.7,62.0
4,2004-07-05,0,1545,22.8,19.5,26.2,14.0
...,...,...,...,...,...,...,...
5491,2020-04-25,1,7299,13.1,9.9,17.1,0.0
5492,2020-04-26,1,11180,12.3,6.8,19.0,0.0
5493,2020-04-27,0,2104,12.9,8.7,18.3,0.0
5494,2020-04-28,0,2709,13.9,7.8,19.9,0.0


In [7]:
### 공휴일 여부 feature 추가
import requests
from urllib import parse
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def getHoliday(year):
    url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
    api_key_utf8 = 'pWAOD4HUZeIOF1lkKUAJmoxLm7O7OwugDwUfiZLCrBahU%2BApyljQgUkaNCadrIRf5fPExBB8HmNoiN7uyQfOdg%3D%3D'
    api_key_decode = parse.unquote(api_key_utf8)

    params = {
        "ServiceKey" : api_key_decode,
        "solYear" : year,
        "numOfRows" : 100
    }

    response = requests.get(url, params=params)
    xml = BeautifulSoup(response.text, "lxml")
    items = xml.find('items')
    print(items)
    item_list = []
    for item in items:
        item_dict = {
            "이름" : item. find("datename").text.strip(),
            "날짜" : datetime.strptime(item.find("locdate").text.strip(), '%Y%m%d'),
            "공휴일 여부" : 1
        }
        item_list.append(item_dict)

    return pd.DataFrame(item_list)


holiday_df = getHoliday(2004)
for i in range(2005, 2021):
    holiday_df = pd.concat([holiday_df, getHoliday(i)])
holiday_df

<items><item><datekind>01</datekind><datename>신정</datename><isholiday>Y</isholiday><locdate>20040101</locdate><seq>1</seq></item><item><datekind>01</datekind><datename>설날</datename><isholiday>Y</isholiday><locdate>20040121</locdate><seq>1</seq></item><item><datekind>01</datekind><datename>설날</datename><isholiday>Y</isholiday><locdate>20040122</locdate><seq>1</seq></item><item><datekind>01</datekind><datename>설날</datename><isholiday>Y</isholiday><locdate>20040123</locdate><seq>1</seq></item><item><datekind>01</datekind><datename>삼일절</datename><isholiday>Y</isholiday><locdate>20040301</locdate><seq>1</seq></item><item><datekind>01</datekind><datename>식목일</datename><isholiday>Y</isholiday><locdate>20040405</locdate><seq>1</seq></item><item><datekind>01</datekind><datename>어린이날</datename><isholiday>Y</isholiday><locdate>20040505</locdate><seq>1</seq></item><item><datekind>01</datekind><datename>석가탄신일</datename><isholiday>Y</isholiday><locdate>20040526</locdate><seq>1</seq></item><item><dat

,이름,날짜,공휴일 여부
0,신정,2004-01-01,1
1,설날,2004-01-21,1
2,설날,2004-01-22,1
3,설날,2004-01-23,1
4,삼일절,2004-03-01,1
...,...,...,...
13,추석,2020-10-01,1
14,추석,2020-10-02,1
15,개천절,2020-10-03,1
16,한글날,2020-10-09,1


In [8]:
# 필요없는 column 삭제
holiday_df.drop('이름', axis=1, inplace=True)
holiday_df

# merge holiday_df to final_df
final_df = pd.merge(final_df, holiday_df, left_on='날짜', right_on='날짜', how='left')

# 공휴일 여부 : null값 채우기
final_df['공휴일 여부'].fillna(value=0, inplace=True)

final_df

,날짜,주말 여부,총 입장객 수,평균기온(℃),최저기온(℃),최고기온(℃),강수량(mm),공휴일 여부
0,2004-07-01,0,1695,24.0,22.0,26.7,4.5,0.0
1,2004-07-02,0,1240,23.1,21.3,24.2,10.5,0.0
2,2004-07-03,1,3852,24.4,22.0,26.8,10.5,0.0
3,2004-07-04,1,694,21.1,19.7,22.7,62.0,0.0
4,2004-07-05,0,1545,22.8,19.5,26.2,14.0,0.0
...,...,...,...,...,...,...,...,...
5494,2020-04-25,1,7299,13.1,9.9,17.1,0.0,0.0
5495,2020-04-26,1,11180,12.3,6.8,19.0,0.0,0.0
5496,2020-04-27,0,2104,12.9,8.7,18.3,0.0,0.0
5497,2020-04-28,0,2709,13.9,7.8,19.9,0.0,0.0


In [9]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5499 entries, 0 to 5498
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   날짜       5499 non-null   datetime64[ns]
 1   주말 여부    5499 non-null   int64         
 2   총 입장객 수  5499 non-null   int64         
 3   평균기온(℃)  5499 non-null   float64       
 4   최저기온(℃)  5499 non-null   float64       
 5   최고기온(℃)  5499 non-null   float64       
 6   강수량(mm)  5499 non-null   float64       
 7   공휴일 여부   5499 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 386.6 KB


In [10]:
# 날짜 : datetime -> string으로 변환해 DB 저장
# final_df2 = final_df.copy() 
# final_df2['날짜']=final_df2['날짜'].astype(str)
# final_df2.info()
# final_df2.to_excel('/content/Childpark_data_DB.xlsx')
# final_df2.info()

## **ML Model**

In [25]:
final_df.asype

AttributeError: ignored

In [12]:
# pip install category_encoders

In [13]:
# pip install eli5

In [14]:
# 사이킷런 - 데이터 셋 나누기
from sklearn.model_selection import train_test_split

#사이킷런 - 교차검증
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import f_regression, SelectKBest


# 사이킷런 - 파이프라인
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

# 학습모델
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

# 사이킷런 - 모델 평가 지표
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [15]:
# split data into X and y
features = ['주말 여부', '평균기온(℃)', '최저기온(℃)', '최고기온(℃)', '강수량(mm)', '공휴일 여부']
target = '총 입장객 수'

X = final_df[features]
y = final_df[target]


# model : Randomforestregressor
model = RandomForestRegressor().fit(X, y)

In [30]:
import numpy as np

data = pd.DataFrame(np.array([0, 17, 14, 21, 3, 0]).reshape(1, -1))
model.predict(data)

array([4484.66])

In [16]:
# model save
import joblib
joblib.dump(model, 'project3_model_.pkl')

# read
# model = joblib.load("project3_model.pkl")

['project3_model_.pkl']

# etc...

### [xml/json parsing](https://hwi-doc.tistory.com/entry/%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%8F%AC%ED%84%B8-API-%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD-%EA%B5%AD%EA%B2%BD%EC%9D%BC-%EB%B0%8F-%EA%B3%B5%ED%9C%B4%EC%9D%BC)

In [ ]:
# pip install xmltodict

In [ ]:
### 공휴일 API 

# def holi_API(year, month):
#         import requests
#         import json
#         import xmltodict
#         import pprint

#         # url
#         url = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getRestDeInfo'
#         params ={'serviceKey' : 'pWAOD4HUZeIOF1lkKUAJmoxLm7O7OwugDwUfiZLCrBahU+ApyljQgUkaNCadrIRf5fPExBB8HmNoiN7uyQfOdg==', 'numOfRows' : '100', 'solYear' : f'{year}', 'solMonth' : f'{month}'}

#         # 데이터 내용 불러오기
#         response = requests.get(url, params=params)
#         content = response.content
#         print(response)
#         # xmltodict로 파싱
#         content_parsed = xmltodict.parse(content)
#         # pprint.pprint(content_parsed)

#         # xml -> json
#         # 한글 깨짐 방지를 위해 ensure_ascii = False
#         jsonString = json.dumps(content_parsed['response']['body'], ensure_ascii = False)

#         # json -> python
#         jsonObj = json.loads(jsonString)
#         print(jsonObj)
#         # 날짜, 공휴일 여부 추출(공휴일이 없는 경우도 생각해야함)
#         if jsonObj['items'] == None:
#             print('Nan')
#         else:
#             holi_dict = [jsonObj['items']['item']]
#             print(f'holi_dict : \n{holi_dict}')
#             for i in range(len(holi_dict)):
#                 print(i)
#                 print(f'\n {holi_dict[i]}')
#                 if holi_dict[i]['isHoliday'] != 'N':
#                     date_list.append(holi_dict[i]['locdate'])  # 날짜
#                     isHoliday_list.append(holi_dict[i]['isHoliday'])  # 공휴일 여부
#                 print(holi_dict[i]['locdate'])
#                 print(holi_dict[i]['isHoliday'])
        


# date_list = []
# isHoliday_list = []

# for y in range(2019, 2020):
#     for m in range(1, 13):
#         print(y, m)
#         holi_API(y, m)

# print(date_list)
# print(isHoliday_list)

### 동적 변수

In [ ]:
# 2014 데이터 read
# def read(year, firstmonth, lastmonth):
#     for i in range(firstmonth, lastmonth + 1):
#         excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
#         sheetname = f'{i}월'

#         df = pd.read_excel(excel, sheet_name=sheetname)
#         print(df)
#         print(df.iloc[:, :7])
#         df = pd.concat([df_2014_7.iloc[5:-1, 0], df_2014_7.iloc[5:-1, 7:15]], axis=1)
#         print(df_2014_7.iloc[5:-1, 0])
#         print(df_2014_7.iloc[5:-1, 7:15])
#         df.columns = ['날짜', '유료소계', '어른', '청소년',	'어린이', '외국인', '단체입장', '무료소계', '총계']
#         df.reset_index(inplace=True, drop=True)
        
#         # 파이썬 동적 변수
#         globals()['df_{}_{}'.format(year, i)] = df
#         print(globals()['df_{}_{}'.format(year, i)])  # 생성된 df 확인

# 데이터 읽기
# def make_df(year, firstmonth, lastmonth):
#     for i in range(firstmonth, lastmonth + 1):
#         excel = f'/content/서울대공원 입장객 정보_{year}년.xls'
#         month = f'{i}월'
        
#         # 필요한 데이터만 남기기
#         df = pd.read_excel(excel, sheet_name=month)
#         df = pd.concat([df.iloc[5:-1, 0], df.iloc[5:-1, 7:15]], axis=1)
#         df.columns = ['날짜', '유료소계', '어른', '청소년',	'어린이', '외국인', '단체입장', '무료소계', '총계']
#         df.reset_index(inplace=True, drop=True)
#         globals()['df_{}_{}'.format(year, i)] = df
#         return globals()['df_{}_{}'.format(year, i)]

### time-series data split

In [ ]:
# time-series 데이터 셋 나누기 
cutoff = pd.to_datetime('2018-01-01')
train = final_df[final_df['날짜'] < cutoff]
test = final_df[final_df['날짜'] >= cutoff]

# Data leakage 방지-test, train 분리 확인
print(f'train과 test 데이터는 완전히 분리되었습니까? : {set(train.T).isdisjoint(set(test.T))}')


features = ['날짜', '주말 여부', '평균기온(℃)', '최저기온(℃)', '최고기온(℃)', '강수량(mm)', '공휴일 여부']
target = '총 입장객 수'

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

print(f'훈련 데이터 : {X_train.shape},{y_train.shape}')
print(f'테스트 데이터 : {X_test.shape},{y_test.shape}')